
---

# Building an E-commerce Shop Agent Recommendation AI Agent with ADK and Vector Search



### What are Generative Recommendations?

Generative Recommendation enables AI to do more than just match user queries to products. It can interpret the user's intent, conduct external research, and generate new, relevant search queries or suggestions based on context and deeper understanding.

**Key features include:**

1. **Understanding and Expanding User Intent:**  
   The AI can infer the user's underlying needs. For example, if someone asks for a "birthday present for a 10-year-old boy," the system doesn't just search for those exact words.

2. **Using External Research (Google Search):**  
   The AI leverages tools like Google Search to research what items are typically bought for a given purpose, such as popular gifts for 10-year-old boys.

3. **Generating New Queries:**  
   Based on its research, the AI creates a list of more specific and varied search queries, such as "educational toys for 10-year-olds," "adventure books for boys," or "coding kits for kids," to broaden and refine the search results.

In summary, Generative Recommendation goes beyond simple keyword matching. It proactively helps users discover products by generating new ideas and relevant search paths.

**Workflow Overview:**

The agent system you’ll build follows this process:

1. The user asks the Shop Agent for a recommendation.
2. The Shop Agent consults the Research Agent.
3. The Research Agent uses Google Search to gather information and generates several targeted queries.
4. The Shop Agent presents these queries to the user for approval.
5. Upon approval, the Shop Agent uses the queries to search for items via the Vector Search backend.
6. The results are returned to the user, completing the recommendation cycle.

---



## Install ADK

First, we will install the ADK. In Colab Enterprise, you may see `ERROR: pip's dependency resolver does...` but you can ignore it.

In [1]:
%pip install --upgrade google-adk==1.4.2 -q

Note: you may need to restart the kernel to use updated packages.


### Import the libraries

In [2]:
import os 
import logging
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.agent_tool import AgentTool

# Ignore warnings from ADK and Gemini APIs
logging.getLogger("google.adk.runners").setLevel(logging.ERROR)
logging.getLogger("google_genai.types").setLevel(logging.ERROR)



In [3]:
from dotenv import load_dotenv

load_dotenv()
os.environ['GEMINI_API_KEY']=os.getenv('GEMINI_API_KEY')


### Set environment variables

Get an API key from [Google AI Studio](https://aistudio.google.com/apikey), set the key to the following `GOOGLE_API_KEY`, and run the cell to set the environment variables required for running ADK.

## Define **test_agent** function for testing agents

For testing the agents we will build, we need to define a function `test_agent` that uses `Runner` and `SessionService` to emulate an agent runtime environment. To learn more about the agent runtime, see the [Agent Runtime](https://google.github.io/adk-docs/runtime/) doc.

In [4]:
from google.genai import types 

APP_NAME="shop_assitant_app"
USER_ID="user_1"
session_service=InMemorySessionService()

async def test_agent(query,agent):
    
    print(f"\n>> query :{query}")
    
    session=await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID
    )
    
    runner=Runner(
        app_name=APP_NAME,
        agent=agent,
        session_service=session_service
    )
    
    content=types.Content(role='user',parts=[types.Part(text=query)])
    
    final_response_text=None
    async for event in runner.run_async(user_id=USER_ID,session_id=session.id,new_message=content):
        if event.is_final_response():
            if event.content and event.content.parts:
                final_response_text=event.content.parts[0].text 
            break
    print(f"<< Agent response:{final_response_text}")       
                

## Define an **Shop agent**

Let's define a shop agent and test it. Note that this agent does not have any search capability at this time.

In [24]:
instruction=f'''
    Your role is a shop search agent on an e-commerce site with millions of
    items. Your responsibility is to search items based on user queries.
'''

shop_agent = Agent(
   model='gemini-2.5-flash',
    name='shop_agent',
    description=(
        'Shop agent for an e-commerce site'
    ),
    instruction=instruction,
)







In [25]:
await test_agent("What kind of site is this?", shop_agent)



>> query :What kind of site is this?
<< Agent response:This is an e-commerce site where you can search for and buy millions of items.


## Define **call_vector_search** to call the Vector Search backend

With the basic agent above, we would like to add an item search capability. To achieve this, here we define a function `call_query_api` that sends an HTTP request to a REST endpoint provided by the [Vector Search Interactive demo](https://cloud.google.com/vertex-ai/docs/vector-search/try-it). For the detail of each parameter sent to the endpoint, refer to the demo page.

In [30]:
import requests
import json

def call_vector_search(url, query, rows=None):
    """
    Calls the Vector Search backend for querying.

    Args:
        url (str): The URL of the search endpoint.
        query (str): The query string.
        rows (int, optional): The number of result rows to return. Defaults to None.

    Returns:
        dict: The JSON response from the API.
    """

    # Build HTTP headers and a payload
    headers = {'Content-Type': 'application/json'}
    payload = {
        "query": query,
        "rows": rows,
        "dataset_id": "mercari3m_mm", # Use Mercari 3M multimodal index
        "use_dense": True, # Use multimodal search
        "use_sparse": True, # Use keyword search too
        "rrf_alpha": 0.5, # Both results are merged with the same weights
        "use_rerank": True, # Use Ranking API for reranking
    }

    # Send an HTTP request to the search endpoint
    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error calling the API: {e}")
        return None


## Define **find_shopping_items** tool

Now, we will wrap the `call_vector_search` function with an ADK Tool named `find_shopping_items`. Note that we need to 1) use the explicit typing such as `queries: list[str]` and 2) use the verbose docstring, both for conveying the functionality and semantics of this Tool to the agent.

For details of the Tool mechanism of ADK, refer to the [Tools](https://google.github.io/adk-docs/tools/) in the ADK docs.

In [32]:
from typing import Dict

def find_shopping_items(queries: list[str]) -> Dict[str, str]:
    """
    Find shopping items from the e-commerce site with the specified list of
    queries.

    Args:
        queries: the list of queries to run.
    Returns:
        A dict with the following one property:
            - "status": returns the following status:
                - "success": successful execution
            - "items": items found in the e-commerce site.
    """
    url = "https://www.ac0.cloudadvocacyorg.joonix.net/api/query"

    items = []
    for query in queries:
        result = call_vector_search(
            url=url,
            query=query,
            rows=3,
        )
        items.extend(result["items"])

    print("-----")
    print(f"User queries: {queries}")
    print(f"Found: {len(items)} items")
    print("-----")
    return items



Let's test this tool.

In [33]:
find_shopping_items(["Cup with dancing animal","Cups with dancing peeople"])

-----
User queries: ['Cup with dancing animal', 'Cups with dancing peeople']
Found: 6 items
-----


[{'dense_dist': 0.21062757074832916,
  'description': 'Dansk Bistro Christmas mug\nNo cracks or chips\nHand painted graphics on both sides\nPlease see all photos as they are part of the description \n\n\n\n\nPlease note: I ship via the fastest and most reasonable methods that Mercari offers. Especially with fragile items. That being said, please check out my shop for more as I’m happy to bundle and save you on shipping',
  'id': 'm75681278572',
  'img_url': 'https://u-mercari-images.mercdn.net/photos/m75681278572_1.jpg?w=200&h=200&fitcrop&sharpen',
  'name': 'Dansk Bistro Christmas mug -reindeer',
  'rerank_score': 0.0,
  'sparse_dist': None,
  'url': 'https://www.mercari.com/us/item/m75681278572'},
 {'dense_dist': 0.0,
  'description': 'Brand new / dancing figure',
  'id': 'm11326867382',
  'img_url': 'https://u-mercari-images.mercdn.net/photos/m11326867382_1.jpg?w=200&h=200&fitcrop&sharpen',
  'name': 'Hello Kitty Dancing figure',
  'rerank_score': 0.0,
  'sparse_dist': 0.48474636673

## Add the tool to **Shop agent**

It's ready to add the Tool to the Shop agent. The following parts are added:

- Addition to the `instruction`: `To find items use find_shopping_items tool by passing a list of queries, and answer to the user with item's name, description and img_url`
- Adding `tools` parameter to the `Agent` constructor: `tools=[find_shopping_items]`



In [36]:
instruction = f'''
    Your role is a shop search agent on an e-commerce site with millions of
    items. Your responsibility is to search items based on the queries you
    recieve.

    To find items use `find_shopping_items` tool by passing a list of queries,
    and answer to the user with item's name, description and img_url
'''

shop_agent = Agent(
    model='gemini-2.5-flash',
    name='shop_agent',
    description=(
        'Shop agent for an e-commerce site'
    ),
    instruction=instruction,
    tools=[find_shopping_items],
)

Let's test the agent.

In [37]:
await test_agent("Cups with dancing figures",shop_agent)


>> query :Cups with dancing figures
-----
User queries: ['Cups with dancing figures']
Found: 3 items
-----
<< Agent response:Here are a few items that might interest you:

*   **Name:** Mayfair Collection By Jay-African Ladies/Baskets/Sun- Tea/Coffee Cup/Mug
    **Description:** Mayfair Collection By Jay-African Ladies/Baskets/Sun- Tea/Coffee Cup/Mug. The mugs are in very good pre-owned condition with no cracks, chips, crazing, or visible scratches. There may be worn consistent with normal use as this is a preloved item. Please view photos carefully for condition. This item presents as it has not been used much. Dimensions: 4.25" tall x 4.5" wide with handle x 3.25" across NON-SMOKING AND PET-FREE pre-owned Items are washed and professionally cleaned before being mailed out to customers. Please check my other listings for more great items. I am always happy to combine items from multiple purchases whenever possible to save you $$$ on shipping costs. Message me if you have any question

## Define **Research agent** with Google Search grounding

Next, we will define another agent `research_agent`. This agent will take the user query and use the built-in Google Search tool for researching on what kind of items people are purchasing for the user's intent. Then generate 5 queries for finding those items.

Note that the following agent definition specifies `google_search` as a tool. With this, the agent obtains a capability to use Google Search. For details about the Google Search tool, refer to [Google Search](https://google.github.io/adk-docs/tools/built-in-tools/#google-search) on the ADK docs.

In [ ]:
from google.adk.tools import google_search

instruction = f'''
    Your role is a market researcher for an e-commerce site with millions of
    items.

    When you recieved a search request from an user, use Google Search tool to
    research on what kind of items people are purchasing for the user's intent.

    Then, generate 5 queries finding those items on the e-commerce site and
    return them.
'''

research_agent = Agent(
    model='gemini-2.5-flash',
    name='research_agent',
    description=('''
        A market researcher for an e-commerce site. Receives a search request
        from a user, and returns a list of 5 generated queries in English.
    '''),
    instruction=instruction,
    tools=[google_search],
)

Let's test the agent.

In [ ]:
await test_agent("birthday present for my 8 year old daughter",research_agent)

## Finalize the **Shop agent**

To wrap up, we modify the `shop_agent` to use both `reseach_agent` and `find_shopping_items` tool.

In [40]:
instruction= f'''
    Your role is a shopper's concierge for an e-commerce site with millions of
    items. Follow the following steps.

    When you recieved a search request from an user, pass it to `research_agent`
    tool, and receive 5 generated queries. Then, pass the list of queries to
    `find_shopping_items` to find items. When you recieved a list of items from
    the tool, answer to the user with item's name, description and the image url.
'''

shop_agent=Agent(
    model='gemini-2.5-flash',
    name='shop_agent',
    description=(
        'A shopper concierge for an e-commerce site'
    ),
    instruction=instruction,
    tools=[
        AgentTool(agent=research_agent),
        find_shopping_items,
    ]
)

Let's test the agent. First, the user asks the agent for finding items. The Shop agent will call the Research agent for generating queries using Google Search Results.

In [42]:
await test_agent("Can you find birthday present for my 8 year old daughter?",shop_agent)


>> query :Can you find birthday present for my 8 year old daughter?
-----
User queries: ['Craft and jewelry making kits for 8 year old girls', 'STEM science kits for 8 year old girls', 'LEGO sets for 8 year old girls', 'Outdoor toys and sports equipment for 8 year old girls', 'Popular books and graphic novels for 8 year old girls']
Found: 15 items
-----
<< Agent response:Here are some birthday present ideas for your 8-year-old daughter:

*   **Craft kits**
    Description: 2 craft kits. Includes fabric and instructions for projects
    Image: https://u-mercari-images.mercdn.net/photos/m54900134619_1.jpg?w=200&h=200&fitcrop&sharpen

*   **⭐ 6750 Pcs Clay Beads Kit for Bracelet Making, 28 Colors ⭐**
    Description: 【PRODUCT DESCRIPTION】Our team pays great attention to the matching of beads in the kit, so we spend a long time to choose the type of beads in our products. Also, this helps our customers to create more beautiful jewelry that is different from other products and makes DIY ea

## Summary

In this tutorial, we went through the process of constructing a multi-agent system for an e-commerce platform, focusing on "Generative Recommendations".

Through this progression, the tutorial illustrated how to build a sophisticated AI agent system that can understand user intent, perform research, generate targeted queries, and provide relevant product recommendations in an e-commerce context using ADK and external search capabilities.
